In [13]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

from tqdm import tqdm

import matplotlib.pyplot as plt
import lib.draw as draw
import imageio

In [14]:
df = pd.read_csv('data/ronaldo_calma.csv')
df.edgecolor.fillna('white', inplace=True)
df.bgcolor.fillna('black', inplace=True)
df.head()

,player,frame,x,y,z,dx,dy,coords,player_num,player_obj,team,num,name,edgecolor,bgcolor
0,0,0,38.217627,15.961068,0.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,white,black
1,0,1,37.447818,15.742122,0.0,-0.769809,-0.218947,NaN,NaN,NaN,NaN,NaN,NaN,white,black
2,0,2,36.729118,15.537712,0.0,-0.718700,-0.204410,NaN,NaN,NaN,NaN,NaN,NaN,white,black
3,0,3,36.058134,15.346873,0.0,-0.670984,-0.190839,NaN,NaN,NaN,NaN,NaN,NaN,white,black
4,0,4,35.431698,15.168704,0.0,-0.626436,-0.178169,NaN,NaN,NaN,NaN,NaN,NaN,white,black


In [15]:
smoothing = 5
temp_file_name = '_tmp/002_tmp_mine.png'

In [16]:
grid = pd.DataFrame([[i/1.05, j/0.68, i/1.05, j/0.68, i/1.05, j/0.68] for i in range(106) for j in range(69)], columns=['x','y','x_1dx','y_1dy','x_2dx','y_2dy'])
delay_0_model = KNeighborsClassifier(n_neighbors=1, n_jobs=-1)
delay_1_model = KNeighborsClassifier(n_neighbors=1, n_jobs=-1)
delay_2_model = KNeighborsClassifier(n_neighbors=1, n_jobs=-1)

In [17]:
writer = imageio.get_writer('ronaldocalma_voronoi.gif', mode='I')
for frame_no in tqdm(df.frame.unique()):
    frame = df[df.frame==frame_no].copy()

    train_set = frame[~frame.team.isna()].copy()
    train_set.team = (train_set.team == 'attack') * 2 - 1

    delay_0_model.fit(train_set[['x','y']], train_set['team']);

    grid['delay_0_control'] = delay_0_model.predict(grid[['x', 'y']])
    grid['control'] = grid['delay_0_control']

    draw.pitch(dpi=144)
    plt.scatter(grid.x, grid.y, s=10, marker='s', c=grid.control, cmap='seismic', alpha=0.2);
    plt.scatter(frame.x, frame.y, s=100, c=frame.bgcolor.values, edgecolors=frame.edgecolor);
    plt.savefig(temp_file_name, bbox_inches='tight')
    plt.clf()
    image = imageio.imread(temp_file_name)
    writer.append_data(image)

100%|██████████| 289/289 [04:34<00:00,  1.05it/s]


<Figure size 1512x979.2 with 0 Axes>

In [18]:
writer = imageio.get_writer('ronaldocalma_delayknn.gif', mode='I')
for frame_no in tqdm(df.frame.unique()):
    frame = df[df.frame==frame_no].copy()

    train_set = frame[~frame.team.isna()].copy()
    train_set.team = (train_set.team == 'attack') * 2 - 1
    train_set['x_1dx'] = train_set.x + 10 * train_set.dx
    train_set['y_1dy'] = train_set.y + 10 * train_set.dy
    train_set['x_2dx'] = train_set.x + 20 * train_set.dx
    train_set['y_2dy'] = train_set.y + 20 * train_set.dy

    delay_0_model.fit(train_set[['x','y']], train_set['team']);
    delay_1_model.fit(train_set[['x_1dx','y_1dy']], train_set['team']);
    delay_2_model.fit(train_set[['x_2dx','y_2dy']], train_set['team']);

    grid['delay_0_control'] = delay_0_model.predict(grid[['x', 'y']])
    grid['delay_1_control'] = delay_1_model.predict(grid[['x_1dx', 'y_1dy']])
    grid['delay_2_control'] = delay_2_model.predict(grid[['x_2dx', 'y_2dy']])

    grid['control'] = grid['delay_0_control'] + grid['delay_1_control'] + grid['delay_2_control']

    draw.pitch(dpi=144)
    for x in grid.x.unique():
        grid.loc[grid.x == x, 'control'] = grid.loc[grid.x == x, 'control'].rolling(smoothing, min_periods=1, center=True).mean()
        grid.loc[grid.y == x, 'control'] = grid.loc[grid.y == x, 'control'].rolling(smoothing, min_periods=1, center=True).mean()
    plt.scatter(grid.x, grid.y, s=10, marker='s', c=grid.control, cmap='seismic', alpha=0.2);
    plt.scatter(frame.x, frame.y, s=100, c=frame.bgcolor.values, edgecolors=frame.edgecolor);
    plt.savefig(temp_file_name, bbox_inches='tight')
    plt.clf()
    image = imageio.imread(temp_file_name)
    writer.append_data(image)

100%|██████████| 289/289 [08:41<00:00,  1.80s/it]


<Figure size 1512x979.2 with 0 Axes>

In [19]:
writer = imageio.get_writer('ronaldocalma_delayknn_2exp_distance.gif', mode='I')
for frame_no in tqdm(df.frame.unique()):
    frame = df[df.frame==frame_no].copy()

    train_set = frame[~frame.team.isna()].copy()
    train_set.team = (train_set.team == 'attack') * 2 - 1
    train_set['x_1dx'] = train_set.x + 10 * train_set.dx
    train_set['y_1dy'] = train_set.y + 10 * train_set.dy
    train_set['x_2dx'] = train_set.x + 20 * train_set.dx
    train_set['y_2dy'] = train_set.y + 20 * train_set.dy

    delay_0_model.fit(train_set[['x','y']], train_set['team']);
    delay_1_model.fit(train_set[['x_1dx','y_1dy']], train_set['team']);
    delay_2_model.fit(train_set[['x_2dx','y_2dy']], train_set['team']);

    grid['delay_0_control'] = delay_0_model.predict(grid[['x', 'y']])
    grid['delay_1_control'] = delay_1_model.predict(grid[['x_1dx', 'y_1dy']])
    grid['delay_2_control'] = delay_2_model.predict(grid[['x_2dx', 'y_2dy']])

    distances, indices = delay_0_model.kneighbors(grid[['x', 'y']])
    factor_d0 = np.exp(np.exp(1 - (distances[:,0] / 40)))
    distances, indices = delay_1_model.kneighbors(grid[['x_1dx', 'y_1dy']])
    factor_d1 = np.exp(np.exp(1 - (distances[:,0] / 40)))
    distances, indices = delay_2_model.kneighbors(grid[['x_2dx', 'y_2dy']])
    factor_d2 = np.exp(np.exp(1 - (distances[:,0] / 40)))

    grid['control'] = grid['delay_0_control'] * factor_d0 + grid['delay_1_control'] * factor_d1 + grid['delay_2_control'] * factor_d2

    draw.pitch(dpi=144)
    for x in grid.x.unique():
        grid.loc[grid.x == x, 'control'] = grid.loc[grid.x == x, 'control'].rolling(smoothing, min_periods=1, center=True).mean()
        grid.loc[grid.y == x, 'control'] = grid.loc[grid.y == x, 'control'].rolling(smoothing, min_periods=1, center=True).mean()
    plt.scatter(grid.x, grid.y, s=10, marker='s', c=grid.control, cmap='seismic', alpha=0.2);
    plt.scatter(frame.x, frame.y, s=100, c=frame.bgcolor.values, edgecolors=frame.edgecolor);
    plt.savefig(temp_file_name, bbox_inches='tight')
    plt.clf()
    image = imageio.imread(temp_file_name)
    writer.append_data(image)

100%|██████████| 289/289 [07:48<00:00,  1.62s/it]


<Figure size 1512x979.2 with 0 Axes>

In [20]:
writer = imageio.get_writer('ronaldocalma_delayknn_3exp_distance.gif', mode='I')
for frame_no in tqdm(df.frame.unique()):
    frame = df[df.frame==frame_no].copy()

    train_set = frame[~frame.team.isna()].copy()
    train_set.team = (train_set.team == 'attack') * 2 - 1
    train_set['x_1dx'] = train_set.x + 10 * train_set.dx
    train_set['y_1dy'] = train_set.y + 10 * train_set.dy
    train_set['x_2dx'] = train_set.x + 20 * train_set.dx
    train_set['y_2dy'] = train_set.y + 20 * train_set.dy

    delay_0_model.fit(train_set[['x','y']], train_set['team']);
    delay_1_model.fit(train_set[['x_1dx','y_1dy']], train_set['team']);
    delay_2_model.fit(train_set[['x_2dx','y_2dy']], train_set['team']);

    grid['delay_0_control'] = delay_0_model.predict(grid[['x', 'y']])
    grid['delay_1_control'] = delay_1_model.predict(grid[['x_1dx', 'y_1dy']])
    grid['delay_2_control'] = delay_2_model.predict(grid[['x_2dx', 'y_2dy']])

    distances, indices = delay_0_model.kneighbors(grid[['x', 'y']])
    factor_d0 = np.exp(np.exp(np.exp(1 - (distances[:,0] / 40))))
    distances, indices = delay_1_model.kneighbors(grid[['x_1dx', 'y_1dy']])
    factor_d1 = np.exp(np.exp(np.exp(1 - (distances[:,0] / 40))))
    distances, indices = delay_2_model.kneighbors(grid[['x_2dx', 'y_2dy']])
    factor_d2 = np.exp(np.exp(np.exp(1 - (distances[:,0] / 40))))

    grid['control'] = grid['delay_0_control'] * factor_d0 + grid['delay_1_control'] * factor_d1 + grid['delay_2_control'] * factor_d2

    draw.pitch(dpi=144)
    for x in grid.x.unique():
        grid.loc[grid.x == x, 'control'] = grid.loc[grid.x == x, 'control'].rolling(smoothing, min_periods=1, center=True).mean()
        grid.loc[grid.y == x, 'control'] = grid.loc[grid.y == x, 'control'].rolling(smoothing, min_periods=1, center=True).mean()
    plt.scatter(grid.x, grid.y, s=10, marker='s', c=grid.control, cmap='seismic', alpha=0.2);
    plt.scatter(frame.x, frame.y, s=100, c=frame.bgcolor.values, edgecolors=frame.edgecolor);
    plt.savefig(temp_file_name, bbox_inches='tight')
    plt.clf()
    image = imageio.imread(temp_file_name)
    writer.append_data(image)

100%|██████████| 289/289 [07:46<00:00,  1.61s/it]


<Figure size 1512x979.2 with 0 Axes>